In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os, json, math, logging, warnings
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore", category=FutureWarning)

try:
    import yfinance as yf
except Exception:
    yf = None

try:
    import requests
except Exception:
    requests = None

# -------------------- LOGGING --------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
log = logging.getLogger("daily_scheduler")

# -------------------- CONFIG --------------------
@dataclass
class Config:
    # Data
    start_date: str = "2015-01-01"
    end_date: Optional[str] = None              # None -> through today
    static_symbols: Optional[List[str]] = None
    static_symbols_path: Optional[str] = "nifty500.txt"
    cache_dir: str = "cache"
    out_dir: str = "outputs"

    # Historical run
    override_trade_date: Optional[str] = "2016-01-11" #None   # e.g. "2025-10-16"

    # Toggles (mirror backtest)
    use_trend_fast_slow: bool = False
    use_htf_trend: bool       = False
    use_rsi_confirm: bool   = True
    use_macd_confirm: bool  = False
    use_adx_confirm: bool   = False
    use_sma50_confirm: bool = False
    use_bbmid_confirm: bool = False

    use_basic_liquidity: bool = False
    min_price_inr: float = 50.0
    min_avg_vol_20d: float = 50_000.0

    use_52w_filter: bool     = True
    use_volar_ranking: bool  = True
    use_mvo_sizing: bool     = True

    ema_fast: int = 10
    ema_slow: int = 20
    ema_htf:  int = 200

    stop_loss_pct: float = 0.05
    target_pct:    float = 0.10

    apply_fees: bool       = True
    initial_capital: float = 500_000.0
    max_concurrent_positions: int = 5
    deploy_cash_frac: float = 0.25
    top_k_daily: int = 300

    benchmark_try: Tuple[str,...] = ("^CNX500","^CRSLDX","^NSE500","^NIFTY500","^BSE500","^NSEI")
    volar_lookback: int = 252
    filter_52w_window: int = 252
    within_pct_of_52w_high: float = 0.50

    positions_csv: str = "portfolio_positions.csv"
    trades_log_csv: str = "trades_log.csv"
    account_state_path: str = "account_state.json"

    # Telegram (optional)
    enable_telegram: bool = False
    telegram_bot_token: str = ""
    telegram_chat_id: str = ""

CFG = Config()

# -------------------- FEES --------------------
APPLY_FEES = True
def calc_fees(turnover_buy: float, turnover_sell: float) -> float:
    if not APPLY_FEES: return 0.0
    BROKER_PCT = 0.001; BROKER_MIN = 5.0; BROKER_CAP = 20.0
    STT_PCT = 0.001; STAMP_BUY_PCT = 0.00015
    EXCH_PCT = 0.0000297; SEBI_PCT = 0.000001; IPFT_PCT = 0.000001
    GST_PCT = 0.18; DP_SELL = 20.0 if turnover_sell >= 100 else 0.0
    def _broker(turnover):
        if turnover <= 0: return 0.0
        fee = turnover * BROKER_PCT
        return max(BROKER_MIN, min(fee, BROKER_CAP))
    br_buy = _broker(turnover_buy); br_sell = _broker(turnover_sell)
    stt = STT_PCT * (turnover_buy + turnover_sell)
    stamp = STAMP_BUY_PCT * turnover_buy
    exch = EXCH_PCT * (turnover_buy + turnover_sell)
    sebi = SEBI_PCT * (turnover_buy + turnover_sell)
    ipft = IPFT_PCT * (turnover_buy + turnover_sell)
    dp = DP_SELL
    gst_base = br_buy + br_sell + dp + exch + sebi + ipft
    gst = GST_PCT * gst_base
    return float((br_buy + br_sell) + stt + stamp + exch + sebi + ipft + dp + gst)

# -------------------- HELPERS --------------------
def ensure_dirs(*paths):
    for p in paths: os.makedirs(p, exist_ok=True)

def today_str():
    return pd.Timestamp.today(tz="Asia/Kolkata").strftime("%Y-%m-%d")

def load_static_symbols(static_symbols: Optional[List[str]], static_symbols_path: Optional[str]) -> List[str]:
    if static_symbols and len(static_symbols) > 0:
        syms = list(static_symbols)
    elif static_symbols_path and os.path.exists(static_symbols_path):
        with open(static_symbols_path, "r") as f:
            syms = [line.strip() for line in f if line.strip()]
    else:
        raise ValueError("Provide CFG.static_symbols or CFG.static_symbols_path.")
    out = []
    for s in syms:
        s = s.strip().upper()
        if not s.endswith(".NS"): s = f"{s}.NS"
        out.append(s)
    uniq, seen = [], set()
    for s in out:
        if s not in seen:
            uniq.append(s); seen.add(s)
    return uniq

def fetch_prices(tickers: List[str], start: str, end: Optional[str], cache_dir: str) -> Dict[str, pd.DataFrame]:
    """yfinance end is exclusive -> request end+1d; fix cache freshness vs desired_last."""
    ensure_dirs(cache_dir)
    data = {}
    if end:
        desired_last = pd.to_datetime(end).normalize()
        end_excl = (desired_last + pd.Timedelta(days=1)).strftime("%Y-%m-%d")
    else:
        today_ist = pd.Timestamp.today(tz="Asia/Kolkata").normalize()
        desired_last = today_ist
        end_excl = (today_ist + pd.Timedelta(days=1)).strftime("%Y-%m-%d")
    for ticker in tickers:
        cache_path = os.path.join(cache_dir, f"{ticker.replace('^','_')}.parquet")
        if os.path.exists(cache_path):
            try:
                df = pd.read_parquet(cache_path)
                if len(df):
                    last_in_cache = pd.to_datetime(df.index[-1]).normalize()
                    if last_in_cache >= desired_last:
                        data[ticker] = df; continue
            except Exception:
                pass
        try:
            df = yf.download(
                ticker, start=start, end=end_excl,
                auto_adjust=True, progress=False, multi_level_index=False,
            )
            if df is None or df.empty: continue
            df = df.rename(columns=str.title)[['Open','High','Low','Close','Volume']].dropna()
            df.index.name = "date"
            df.to_parquet(cache_path)
            data[ticker] = df
        except Exception as e:
            log.warning("Download failed %s: %s", ticker, e); continue
    return data

def send_telegram(text: str, cfg: Config):
    if not cfg.enable_telegram: return
    if not requests:
        log.warning("requests not available; cannot send Telegram"); return
    if not cfg.telegram_bot_token or not cfg.telegram_chat_id:
        log.warning("Telegram token/chat_id missing; skip"); return
    try:
        url = f"https://api.telegram.org/bot{cfg.telegram_bot_token}/sendMessage"
        payload = {"chat_id": cfg.telegram_chat_id, "text": text, "parse_mode": "HTML", "disable_web_page_preview": True}
        requests.post(url, json=payload, timeout=10)
    except Exception as e:
        log.warning("Telegram send failed: %s", e)

# -------------------- INDICATORS & PATTERNS --------------------
def sma(series: pd.Series, window: int) -> pd.Series:
    return series.rolling(window, min_periods=window).mean()

def ema(series: pd.Series, span: int) -> pd.Series:
    return series.ewm(span=span, adjust=False, min_periods=span).mean()

def rsi(series: pd.Series, length: int = 14) -> pd.Series:
    delta = series.diff()
    gain = (delta.where(delta > 0, 0.0)).rolling(length).mean()
    loss = (-delta.where(delta < 0, 0.0)).rolling(length).mean()
    rs = gain / loss.replace(0.0, np.nan)
    return (100 - (100 / (1 + rs))).fillna(50.0)

def macd(series: pd.Series, fast=12, slow=26, signal=9):
    ef = ema(series, fast); es = ema(series, slow)
    line = ef - es; sig = ema(line, signal)
    return line, sig, line - sig

def _true_range(high: pd.Series, low: pd.Series, prev_close: pd.Series) -> pd.Series:
    return pd.concat([(high - low).abs(), (high - prev_close).abs(), (low - prev_close).abs()], axis=1).max(axis=1)

def adx_plus_minus_di(high: pd.Series, low: pd.Series, close: pd.Series, length: int = 14):
    prev_high = high.shift(1); prev_low = low.shift(1); prev_close = close.shift(1)
    up_move = high - prev_high; down_move = prev_low - low
    plus_dm = up_move.where((up_move > down_move) & (up_move > 0), 0.0)
    minus_dm = down_move.where((down_move > up_move) & (down_move > 0), 0.0)
    tr = _true_range(high, low, prev_close)
    alpha = 1.0 / length
    atr = tr.ewm(alpha=alpha, adjust=False, min_periods=length).mean()
    plus_di  = 100 * (plus_dm.ewm(alpha=alpha, adjust=False, min_periods=length).mean() / atr)
    minus_di = 100 * (minus_dm.ewm(alpha=alpha, adjust=False, min_periods=length).mean() / atr)
    dx = 100 * (plus_di - minus_di).abs() / (plus_di + minus_di).replace(0, np.nan)
    adx_series = dx.ewm(alpha=alpha, adjust=False, min_periods=length).mean()
    return adx_series, plus_di, minus_di

def _body(o, c): return (c - o).abs()
def _is_bull(o, c): return c > o
def _is_bear(o, c): return c < o

def patt_bull_engulfing(df: pd.DataFrame) -> pd.Series:
    o, c = df["Open"], df["Close"]; op, cp = o.shift(1), c.shift(1)
    return (_is_bear(op, cp) & _is_bull(o, c) & (c >= op) & (o <= cp)).fillna(False)

def patt_piercing(df: pd.DataFrame) -> pd.Series:
    o, h, l, c = df["Open"], df["High"], df["Low"], df["Close"]
    op, _, lp, cp = o.shift(1), h.shift(1), l.shift(1), c.shift(1)
    mid_prev = (op + cp) / 2.0
    return (_is_bear(op, cp) & _is_bull(o, c) & (o < lp) & (c > mid_prev) & (c < op)).fillna(False)

def patt_morning_star(df: pd.DataFrame) -> pd.Series:
    o, c = df["Open"], df["Close"]; o1, c1 = o.shift(1), c.shift(1); o2, c2 = o.shift(2), c.shift(2)
    bear1 = _is_bear(o2, c2); small2 = (_body(o1, c1) <= (_body(o2, c2) * 0.6)); bull3 = _is_bull(o, c)
    mid1 = (o2 + c2) / 2.0; retrace = c > mid1
    return (bear1 & small2 & bull3 & retrace).fillna(False)

def patt_harami_bull(df: pd.DataFrame) -> pd.Series:
    o, c = df["Open"], df["Close"]; o1, c1 = o.shift(1), c.shift(1)
    prev_bear = _is_bear(o1, c1)
    body_small = (_body(o, c) <= _body(o1, c1) * 0.75)
    inside = (np.maximum(o, c) <= o1) & (np.minimum(o, c) >= c1)
    return (prev_bear & body_small & inside & (c >= o)).fillna(False)

def patt_harami_cross_bull(df: pd.DataFrame, doji_pct=0.1) -> pd.Series:
    o, c, h, l = df["Open"], df["Close"], df["High"], df["Low"]; o1, c1 = o.shift(1), c.shift(1)
    rng = (h - l).replace(0, np.nan); doji = (_body(o, c) <= (rng * doji_pct))
    prev_bear = _is_bear(o1, c1); inside = (np.maximum(o, c) <= o1) & (np.minimum(o, c) >= c1)
    return (prev_bear & doji & inside).fillna(False)

def patt_hammer(df: pd.DataFrame, shadow_mult=2.0) -> pd.Series:
    o, c, h, l = df["Open"], df["Close"], df["High"], df["Low"]
    body = _body(o, c); lower_shadow = (np.minimum(o, c) - l).abs(); upper_shadow = (h - np.maximum(o, c)).abs()
    return ((lower_shadow >= shadow_mult * body) & (upper_shadow <= body) & (c >= o)).fillna(False)

def patt_inverted_hammer(df: pd.DataFrame, shadow_mult=2.0) -> pd.Series:
    o, c, h, l = df["Open"], df["Close"], df["High"], df["Low"]
    body = _body(o, c); upper_shadow = (h - np.maximum(o, c)).abs(); lower_shadow = (np.minimum(o, c) - l).abs()
    return ((upper_shadow >= shadow_mult * body) & (lower_shadow <= body) & (c >= o)).fillna(False)

BULLISH_PATTERNS = ["ENGULFING","PIERCING","MORNING_STAR","HARAMI","HARAMI_CROSS","HAMMER","INVERTED_HAMMER"]

def compute_indicators(df: pd.DataFrame, cfg: Config) -> pd.DataFrame:
    out = df.copy()
    out["ema_fast"] = ema(out["Close"], cfg.ema_fast)
    out["ema_slow"] = ema(out["Close"], cfg.ema_slow)
    out["ema_htf"]  = ema(out["Close"], cfg.ema_htf)
    out["rsi"] = rsi(out["Close"], 14)
    macd_line, macd_sig, _ = macd(out["Close"], 12, 26, 9)
    out["macd_line"] = macd_line; out["macd_signal"] = macd_sig
    out["sma50"] = sma(out["Close"], 50); out["bb_mid"] = sma(out["Close"], 20)
    adxv, pdi, ndi = adx_plus_minus_di(out["High"], out["Low"], out["Close"], 14)
    out["adx"] = adxv; out["+di"] = pdi; out["-di"] = ndi
    out["avg_vol_20"] = out["Volume"].rolling(20).mean()
    out["high_52w"] = out["Close"].rolling(cfg.filter_52w_window).max()

    patt_map = {
        "ENGULFING":       patt_bull_engulfing(out),
        "PIERCING":        patt_piercing(out),
        "MORNING_STAR":    patt_morning_star(out),
        "HARAMI":          patt_harami_bull(out),
        "HARAMI_CROSS":    patt_harami_cross_bull(out),
        "HAMMER":          patt_hammer(out),
        "INVERTED_HAMMER": patt_inverted_hammer(out),
    }
    patt_any = None
    for name in BULLISH_PATTERNS:
        patt_any = patt_map[name] if patt_any is None else (patt_any | patt_map[name])
    out["bullish_pattern"] = patt_any.fillna(False)
    return out.dropna()

def _and_if_enabled(enabled: bool, cond: pd.Series) -> pd.Series:
    return cond.astype(bool).fillna(False) if enabled else pd.Series(True, index=cond.index)

def basic_liquidity_ok(row: pd.Series, cfg: Config) -> bool:
    if not cfg.use_basic_liquidity: return True
    if row["Close"] < cfg.min_price_inr: return False
    if row["avg_vol_20"] < cfg.min_avg_vol_20d: return False
    return True

# -------------------- VOLAR & SIZING --------------------
def pick_benchmark(benchmarks: Tuple[str,...], start: str, end: Optional[str], cache_dir: str) -> Tuple[str, pd.DataFrame]:
    for t in benchmarks:
        data = fetch_prices([t], start, end, cache_dir)
        df = data.get(t)
        if df is not None and not df.empty:
            log.info("Using benchmark: %s", t); return t, df
    idx = pd.date_range(start=start, end=end or today_str(), freq="B")
    df = pd.DataFrame({"Close": np.ones(len(idx))}, index=idx)
    log.warning("No benchmark found; using synthetic flat series.")
    return "SYNTH_BENCH", df

def compute_volar_scores(end_dt: pd.Timestamp, tickers: List[str], data_map: Dict[str,pd.DataFrame], bench_df: pd.DataFrame, lookback: int) -> Dict[str, float]:
    scores = {}
    bser = bench_df["Close"].loc[:end_dt].pct_change().dropna().iloc[-lookback:]
    for t in tickers:
        df = data_map.get(t)
        if df is None or df.empty: scores[t] = 0.0; continue
        if end_dt not in df.index:
            df = df[df.index <= end_dt]
            if df.empty: scores[t] = 0.0; continue
        r = df["Close"].loc[:end_dt].pct_change().dropna().iloc[-lookback:]
        common = pd.concat([r, bser], axis=1, keys=["s","b"]).dropna()
        if common.shape[0] < max(20, int(0.4*lookback)):
            scores[t] = 0.0; continue
        excess = common["s"] - common["b"]; vol = common["s"].std(ddof=0)
        scores[t] = 0.0 if vol <= 1e-8 else float((excess.mean() / vol) * math.sqrt(252.0))
    return scores

def markowitz_long_only(mu: np.ndarray, Sigma: np.ndarray) -> np.ndarray:
    n = len(mu); eps = 1e-6
    Sigma = Sigma + eps*np.eye(n)
    def solve_lambda(lmbd: float, active_mask=None):
        if active_mask is None:
            A = np.block([[2*lmbd*Sigma, np.ones((n,1))],[np.ones((1,n)), np.zeros((1,1))]])
            b = np.concatenate([mu, np.array([1.0])])
            try: w = np.linalg.solve(A, b)[:n]
            except np.linalg.LinAlgError: w = np.full(n, 1.0/n)
            return w
        else:
            idx = np.where(active_mask)[0]
            if len(idx)==0: return np.full(n, 1.0/n)
            S = Sigma[np.ix_(idx, idx)]
            o = np.ones(len(idx)); m = mu[idx]
            A = np.block([[2*lmbd*S, o[:,None]],[o[None,:], np.zeros((1,1))]])
            b = np.concatenate([m, np.array([1.0])])
            try: w_sub = np.linalg.solve(A, b)[:len(idx)]
            except np.linalg.LinAlgError: w_sub = np.full(len(idx), 1.0/len(idx))
            w = np.zeros(n); w[idx] = w_sub; return w
    best_w = np.full(n, 1.0/n); best_sr = -1e9
    for lmbd in np.logspace(-3, 3, 31):
        active = np.ones(n, dtype=bool); w = None
        for _ in range(n):
            w = solve_lambda(lmbd, active_mask=active)
            if not (w < 0).any(): break
            active[np.argmin(w)] = False
        if w is None: continue
        w = np.clip(w, 0, None)
        if w.sum() <= 0: continue
        w = w / w.sum()
        mu_p = float(mu @ w); vol_p = float(np.sqrt(w @ Sigma @ w))
        if vol_p <= 1e-8: continue
        sr = mu_p / vol_p
        if sr > best_sr: best_w = w.copy(); best_sr = sr
    return best_w

# -------------------- DATE UTIL --------------------
def next_bar_date_for_ticker(df: pd.DataFrame, run_dt: pd.Timestamp) -> Optional[pd.Timestamp]:
    if df is None or df.empty: return None
    idx = df.index
    pos = idx.searchsorted(run_dt, side="right")
    return idx[pos] if pos < len(idx) else None

# -------------------- SIGNALS (NO FILTERS/CAPS HERE) --------------------
def build_eod_signals_for_date(scan_dt: pd.Timestamp,
                               symbols: List[str],
                               data_map: Dict[str, pd.DataFrame],
                               cfg: Config) -> pd.DataFrame:
    """Emit ALL valid signals on scan_dt. Do NOT 52w-filter or slot-cap here."""
    recs = []
    for tkr in symbols:
        df = data_map.get(tkr)
        if df is None or df.empty or scan_dt not in df.index: continue
        d = compute_indicators(df.loc[:scan_dt], cfg)
        if d.empty or scan_dt not in d.index: continue
        row = d.loc[scan_dt]
        patt_ok = bool(row["bullish_pattern"])
        conds = []
        conds.append(_and_if_enabled(cfg.use_trend_fast_slow, (d["ema_fast"] > d["ema_slow"])))
        conds.append(_and_if_enabled(cfg.use_htf_trend, (d["Close"] > d["ema_htf"])))
        conds.append(_and_if_enabled(cfg.use_rsi_confirm,  (d["rsi"] > 50.0)))
        conds.append(_and_if_enabled(cfg.use_macd_confirm, (d["macd_line"] > d["macd_signal"])))
        conds.append(_and_if_enabled(cfg.use_adx_confirm,  ((d["adx"] > 20.0) & (d["+di"] > d["-di"]))))
        conds.append(_and_if_enabled(cfg.use_sma50_confirm,(d["Close"] > d["sma50"])))
        conds.append(_and_if_enabled(cfg.use_bbmid_confirm,(d["Close"] > d["bb_mid"])))
        if cfg.use_basic_liquidity:
            liq_ok = bool(basic_liquidity_ok(row, cfg))
            conds.append(pd.Series(liq_ok, index=d.index))
        all_ok = True
        for c in conds: all_ok = all_ok & bool(c.loc[scan_dt])
        if not (patt_ok and all_ok): continue

        patt_map = {
            "ENGULFING":       patt_bull_engulfing(d),
            "PIERCING":        patt_piercing(d),
            "MORNING_STAR":    patt_morning_star(d),
            "HARAMI":          patt_harami_bull(d),
            "HARAMI_CROSS":    patt_harami_cross_bull(d),
            "HAMMER":          patt_hammer(d),
            "INVERTED_HAMMER": patt_inverted_hammer(d),
        }
        patterns = [nm for nm, ser in patt_map.items() if bool(ser.loc[scan_dt])]
        patt_str = " + ".join(patterns) if patterns else "BullishPattern"

        sigs = []
        if cfg.use_trend_fast_slow and bool((d["ema_fast"] > d["ema_slow"]).loc[scan_dt]): sigs.append("EMAfast>EMAslow")
        if cfg.use_htf_trend and bool((d["Close"] > d["ema_htf"]).loc[scan_dt]):           sigs.append("Close>EMA200")
        if cfg.use_rsi_confirm and bool((d["rsi"] > 50).loc[scan_dt]):                     sigs.append("RSI>50")
        if cfg.use_macd_confirm and bool((d["macd_line"] > d["macd_signal"]).loc[scan_dt]):sigs.append("MACD>Signal")
        if cfg.use_adx_confirm and bool(((d["adx"] > 20) & (d["+di"] > d["-di"])).loc[scan_dt]): sigs.append("ADX>20 & +DI>-DI")
        if cfg.use_sma50_confirm and bool((d["Close"] > d["sma50"]).loc[scan_dt]):         sigs.append("Close>SMA50")
        if cfg.use_bbmid_confirm and bool((d["Close"] > d["bb_mid"]).loc[scan_dt]):        sigs.append("Close>BBmid")
        if cfg.use_basic_liquidity and bool(basic_liquidity_ok(row, cfg)):                 sigs.append("LiquidityOK")

        reason = "; ".join([f"Entry: {patt_str} + TogglesOK", f"Conf=[{', '.join(sigs) if sigs else 'pattern_only'}]"])

        recs.append({"date": scan_dt, "ticker": tkr, "pattern": patt_str, "reason": reason})

    recs_df = pd.DataFrame(recs)
    if recs_df.empty: return recs_df
    return recs_df.reset_index(drop=True)

# -------------------- POSITIONS / ACCOUNT I/O --------------------
POS_COLS = [
    "ticker","status","planned_entry_date","entry_date","entry_px","qty",
    "stop_px","tgt_px","reason","volar","mvo_weight","alloc_inr","buy_fee","notes"
]

def load_positions(path: str) -> pd.DataFrame:
    if os.path.exists(path):
        df = pd.read_csv(path)
        for c in POS_COLS:
            if c not in df.columns: df[c] = np.nan
        df = df[POS_COLS]
        df["planned_entry_date"] = pd.to_datetime(df["planned_entry_date"], errors="coerce")
        df["entry_date"] = pd.to_datetime(df["entry_date"], errors="coerce")
        return df
    return pd.DataFrame(columns=POS_COLS)

def save_positions(df: pd.DataFrame, path: str):
    df[POS_COLS].to_csv(path, index=False)

def append_trades_log(path: str, rows: List[dict]):
    cols = ["ticker","entry_date","entry_px","qty","exit_date","exit_px","exit_reason",
            "gross_pnl_inr","fees_total_inr","net_pnl_inr","days_held","notes"]
    if not rows: return
    new_df = pd.DataFrame(rows, columns=cols)
    if os.path.exists(path):
        old = pd.read_csv(path); out = pd.concat([old, new_df], ignore_index=True)
    else:
        out = new_df
    out.to_csv(path, index=False)

def load_account_state(path: str, initial_cash: float) -> dict:
    if os.path.exists(path):
        with open(path, "r") as f:
            return json.load(f)
    return {"cash": float(initial_cash)}

def save_account_state(path: str, state: dict):
    with open(path, "w") as f:
        json.dump(state, f, indent=2)

# -------------------- DATE HELPERS --------------------
def find_latest_trade_date_from_map(data_map: Dict[str, pd.DataFrame]) -> Optional[pd.Timestamp]:
    for df in data_map.values():
        if df is not None and not df.empty:
            return df.index[-1]
    return None

def find_trade_date_at_or_before(target: pd.Timestamp, data_map: Dict[str, pd.DataFrame]) -> Optional[pd.Timestamp]:
    dates = []
    for df in data_map.values():
        if df is None or df.empty: continue
        dts = df.index[df.index <= target]
        if len(dts): dates.append(dts[-1])
    return max(dates) if dates else None

# -------------------- DAILY PIPELINE --------------------
def run_daily(cfg: Config):
    ensure_dirs(cfg.cache_dir, cfg.out_dir)
    global APPLY_FEES
    APPLY_FEES = bool(cfg.apply_fees)

    # Universe + Data
    symbols = load_static_symbols(cfg.static_symbols, cfg.static_symbols_path)
    log.info("Universe size: %d", len(symbols))
    data_map = fetch_prices(symbols, cfg.start_date, cfg.end_date, cfg.cache_dir)
    if not data_map:
        log.error("No data fetched. Abort."); return

    # Determine run date
    if cfg.override_trade_date:
        target = pd.to_datetime(cfg.override_trade_date)
        run_dt = find_trade_date_at_or_before(target, data_map)
        if run_dt is None:
            log.error("No trading day on/before %s found in data.", target.date()); return
        log.info("Historical EOD: using %s", run_dt.date())
    else:
        run_dt = find_latest_trade_date_from_map(data_map)
        if run_dt is None:
            log.error("Could not determine latest trading date."); return
        log.info("Live EOD: latest trading date %s", run_dt.date())

    # Benchmark (single instance for the run)
    _, bench_df = pick_benchmark(cfg.benchmark_try, cfg.start_date, cfg.end_date, cfg.cache_dir)

    # Load positions & account
    pos = load_positions(cfg.positions_csv)
    acct = load_account_state(cfg.account_state_path, cfg.initial_capital)
    cash = float(acct.get("cash", cfg.initial_capital))

    # 1) SELL first (TP > SL)
    exits_to_log = []
    open_mask = (pos["status"]=="open")
    for i, r in pos[open_mask].iterrows():
        tkr = r["ticker"]; df = data_map.get(tkr)
        if df is None or df.empty or run_dt not in df.index: continue
        bar = df.loc[run_dt]
        low = float(bar["Low"]); high = float(bar["High"])
        entry_px = float(r["entry_px"]); qty = int(r["qty"])
        stop_px = float(r["stop_px"]);  tgt_px = float(r["tgt_px"])
        entry_fee = float(r.get("buy_fee", 0.0)) if not pd.isna(r.get("buy_fee", np.nan)) else 0.0

        exit_reason = None; exit_px = None
        if (low <= stop_px) and (high >= tgt_px):
            exit_reason = "TakeProfit hit"; exit_px = tgt_px
        elif low <= stop_px:
            exit_reason = "StopLoss hit"; exit_px = stop_px
        elif high >= tgt_px:
            exit_reason = "TakeProfit hit"; exit_px = tgt_px
        else:
            d = compute_indicators(df.loc[:run_dt], cfg)
            if run_dt in d.index and bool((d.loc[run_dt, "ema_fast"] < d.loc[run_dt, "ema_slow"])):
                notes = str(r.get("notes",""))
                if "exit_pending_next_open" not in notes:
                    pos.loc[i, "notes"] = (notes + " | exit_pending_next_open").strip(" |")
                continue

        if exit_reason is not None:
            # SELL proceeds & fees
            turn_sell = exit_px * qty
            sell_fee = calc_fees(0.0, turn_sell)
            cash += (turn_sell - sell_fee)  # add proceeds net of SELL fee
            gross = (exit_px - entry_px) * qty
            total_fees = entry_fee + sell_fee
            net = gross - total_fees
            days_held = int((pd.Timestamp(run_dt) - pd.Timestamp(r["entry_date"])).days)
            exits_to_log.append({
                "ticker": tkr, "entry_date": pd.Timestamp(r["entry_date"]).strftime("%Y-%m-%d"),
                "entry_px": float(entry_px), "qty": int(qty),
                "exit_date": pd.Timestamp(run_dt).strftime("%Y-%m-%d"),
                "exit_px": float(exit_px), "exit_reason": exit_reason,
                "gross_pnl_inr": float(gross), "fees_total_inr": float(total_fees), "net_pnl_inr": float(net),
                "days_held": int(days_held), "notes": str(r.get("reason",""))
            })
            pos = pos.drop(index=i)
            send_telegram(f"🔔 <b>EXIT</b> {tkr}\n{exit_reason} @ {exit_px:.2f} on {run_dt.date()}", cfg)

    if exits_to_log:
        append_trades_log(cfg.trades_log_csv, exits_to_log)
        log.info("Logged %d exits | cash=%.2f", len(exits_to_log), cash)

    # 2) Delayed exits (trend invalidation → next open)
    delayed_mask = (pos["status"]=="open") & (pos["notes"].fillna("").str.contains("exit_pending_next_open"))
    delayed_rows = pos[delayed_mask].copy()
    delayed_logs = []
    for i, r in delayed_rows.iterrows():
        tkr = r["ticker"]; df = data_map.get(tkr)
        if df is None or df.empty or run_dt not in df.index: continue
        open_px = float(df.loc[run_dt, "Open"])
        entry_px = float(r["entry_px"]); qty = int(r["qty"])
        entry_fee = float(r.get("buy_fee", 0.0)) if not pd.isna(r.get("buy_fee", np.nan)) else 0.0
        turn_sell = open_px * qty
        sell_fee = calc_fees(0.0, turn_sell)
        cash += (turn_sell - sell_fee)
        gross = (open_px - entry_px) * qty
        net = gross - (entry_fee + sell_fee)
        days_held = int((pd.Timestamp(run_dt) - pd.Timestamp(r["entry_date"])).days)
        delayed_logs.append({
            "ticker": tkr, "entry_date": pd.Timestamp(r["entry_date"]).strftime("%Y-%m-%d"),
            "entry_px": float(entry_px), "qty": int(qty),
            "exit_date": pd.Timestamp(run_dt).strftime("%Y-%m-%d"),
            "exit_px": float(open_px), "exit_reason": "Trend invalidation -> exited at next open",
            "gross_pnl_inr": float(gross), "fees_total_inr": float(entry_fee + sell_fee), "net_pnl_inr": float(net),
            "days_held": int(days_held), "notes": str(r.get("reason",""))
        })
        pos = pos.drop(index=i)
        send_telegram(f"⚠️ <b>EXIT @ NEXT OPEN</b>\n{tkr} @ {open_px:.2f} on {run_dt.date()}", cfg)

    if delayed_logs:
        append_trades_log(cfg.trades_log_csv, delayed_logs)
        log.info("Logged %d delayed exits | cash=%.2f", len(delayed_logs), cash)

    # 3) Confirm pending BUYs (planned_date <= run_dt)
    pend = pos[(pos["status"]=="pending_entry") & (pos["planned_entry_date"]<=pd.Timestamp(run_dt))].copy()
    if not pend.empty:
        # Exclude already-open
        already_open = set(pos.loc[pos["status"]=="open","ticker"])
        pend = pend[~pend["ticker"].isin(already_open)].copy()

        tickers = pend["ticker"].tolist()
        volar_map = compute_volar_scores(pd.Timestamp(run_dt), tickers, data_map, bench_df, CFG.volar_lookback) if CFG.use_volar_ranking else {t: np.nan for t in tickers}

        filt_rows = []
        for _, rr in pend.iterrows():
            t = rr["ticker"]; df = data_map.get(t)
            if df is None or df.empty or run_dt not in df.index: continue
            # 52w filter ON ENTRY DAY
            hist = df["Close"].loc[:run_dt]
            window = hist.iloc[-CFG.filter_52w_window:] if len(hist)>=CFG.filter_52w_window else hist
            h52 = float(window.max()) if len(window) else np.nan
            close_val = float(df.loc[run_dt, "Close"])
            if CFG.use_52w_filter and not (h52>0 and close_val >= CFG.within_pct_of_52w_high * h52):
                continue
            rr = rr.copy(); rr["volar"] = float(volar_map.get(t, np.nan))
            filt_rows.append(rr)

        pend2 = pd.DataFrame(filt_rows)
        if not pend2.empty and CFG.use_volar_ranking:
            pend2 = pend2.sort_values("volar", ascending=False)

        # Slots and top_k_daily cap (match backtest)
        open_now = pos[pos["status"]=="open"].shape[0]
        slots = max(0, CFG.max_concurrent_positions - open_now)
        cap = min(CFG.top_k_daily, slots)
        to_confirm = pend2.head(cap) if not pend2.empty and cap>0 else pd.DataFrame(columns=pend2.columns)

        # MVO sizing among selected (like backtest)
        weights = np.array([])
        if not to_confirm.empty and CFG.use_mvo_sizing and to_confirm.shape[0] >= 2:
            names = to_confirm["ticker"].tolist()
            R = []
            for t in names:
                ser = data_map[t]["Close"].loc[:run_dt].pct_change().dropna().iloc[-CFG.volar_lookback:]
                R.append(ser)
            R = pd.concat(R, axis=1); R.columns = names; R = R.dropna()
            if R.shape[0] >= max(20, int(0.4*CFG.volar_lookback)):
                mu = R.mean().values; Sigma = R.cov().values
                weights = markowitz_long_only(mu, Sigma)
            else:
                weights = np.full(len(names), 1.0/len(names))
        elif not to_confirm.empty:
            weights = np.full(len(to_confirm), 1.0/len(to_confirm))

        # Cash-based sizing & fee-aware admission
        deploy_cash = max(0.0, float(cash)) * float(CFG.deploy_cash_frac)
        if not to_confirm.empty and deploy_cash <= 0:
            log.info("No deployable cash (cap=%.0f%%). Skipping confirmations.", 100*CFG.deploy_cash_frac)
            to_confirm = pd.DataFrame(columns=to_confirm.columns)

        for k, (_, rr) in enumerate(to_confirm.iterrows()):
            t = rr["ticker"]; df = data_map.get(t)
            if df is None or df.empty or run_dt not in df.index: continue
            open_px = float(df.loc[run_dt, "Open"])
            w = float(weights[k]) if len(weights) else 1.0
            alloc_inr = float((weights.sum() and deploy_cash * w) if len(weights) else deploy_cash)
            if alloc_inr <= 0:
                continue
            qty = int(math.floor(alloc_inr / open_px))
            if qty <= 0:
                continue
            turn_buy = qty * open_px
            buy_fee = calc_fees(turn_buy, 0.0)
            total_cost = turn_buy + buy_fee
            if total_cost > cash:
                # shrink qty to fit cash
                qty = int(math.floor((cash - buy_fee) / open_px))
                if qty <= 0:
                    continue
                turn_buy = qty * open_px
                buy_fee = calc_fees(turn_buy, 0.0)
                total_cost = turn_buy + buy_fee
                if total_cost > cash:
                    continue

            # Admit & deduct cash
            cash -= total_cost
            stop_px = float(open_px * (1 - CFG.stop_loss_pct))
            tgt_px  = float(open_px * (1 + CFG.target_pct))

            idx_in_pos = pos.index[(pos["ticker"]==t) & (pos["status"]=="pending_entry")].tolist()
            if not idx_in_pos: continue
            j = idx_in_pos[0]
            pos.loc[j, "status"] = "open"
            pos.loc[j, "entry_date"] = run_dt
            pos.loc[j, "entry_px"] = open_px
            pos.loc[j, "qty"] = qty
            pos.loc[j, "stop_px"] = stop_px
            pos.loc[j, "tgt_px"] = tgt_px
            pos.loc[j, "mvo_weight"] = (float(weights[k]) if len(weights) else np.nan)
            pos.loc[j, "alloc_inr"] = float(alloc_inr)
            pos.loc[j, "buy_fee"] = float(buy_fee)
            pos.loc[j, "notes"] = (str(pos.loc[j, "notes"]) + " | confirmed@open").strip(" |")

            send_telegram(
                f"✅ <b>ENTRY CONFIRMED</b>\n{t} @ {open_px:.2f} on {run_dt.date()}\n"
                f"Qty: {qty} | SL: {stop_px:.2f} | TP: {tgt_px:.2f}\n{pos.loc[j, 'reason']}", CFG
            )

    # 4) New signals today → plan all for next bar (NO caps/filters/sizing here)
    recos = build_eod_signals_for_date(run_dt, symbols, data_map, CFG)
    held = set(pos["ticker"].tolist())
    if not recos.empty:
        recos = recos[~recos["ticker"].isin(held)].reset_index(drop=True)

    # Plan per-ticker next available bar (no caps)
    new_rows = []
    if not recos.empty:
        for _, r in recos.iterrows():
            t = r["ticker"]; df_t = data_map.get(t)
            nxt_dt = next_bar_date_for_ticker(df_t, run_dt)
            if nxt_dt is None:
                log.info("Skip planning %s (no next bar after %s)", t, run_dt.date()); continue
            new_rows.append({
                "ticker": t, "status": "pending_entry",
                "planned_entry_date": pd.Timestamp(nxt_dt),
                "entry_date": pd.NaT, "entry_px": np.nan, "qty": np.nan,
                "stop_px": np.nan, "tgt_px": np.nan,
                "reason": str(r["reason"]), "volar": np.nan,
                "mvo_weight": np.nan, "alloc_inr": np.nan, "buy_fee": np.nan, "notes": "planned"
            })

        stamp = pd.Timestamp(run_dt).strftime("%Y%m%d")
        recos_out = os.path.join(CFG.out_dir, f"eod_recos_{stamp}.csv")
        recos.to_csv(recos_out, index=False)
        log.info("Wrote recommendations -> %s", recos_out)

        lines = [f"📝 <b>EOD Recos {run_dt.date()}</b> (planned for next bar)"]
        for _, rr in recos.iterrows():
            lines.append(f"• {rr['ticker']}: {rr['pattern']} | {rr['reason']}")
        send_telegram("\n".join(lines[:50]), CFG)

    if new_rows:
        add_df = pd.DataFrame(new_rows, columns=POS_COLS)
        pos = pd.concat([pos, add_df], ignore_index=True)

    # Save positions & account cash
    ensure_dirs(CFG.out_dir)
    save_positions(pos, CFG.positions_csv)
    save_account_state(CFG.account_state_path, {"cash": float(cash)})
    log.info("Positions -> %s | Trades -> %s | Cash=%.2f", CFG.positions_csv, CFG.trades_log_csv, cash)
    log.info("Done.")

def main():
    global APPLY_FEES
    APPLY_FEES = bool(CFG.apply_fees)
    if yf is None:
        log.error("yfinance not available."); return
    run_daily(CFG)

if __name__ == "__main__":
    main()


2025-10-17 22:44:41 | INFO | Universe size: 500
2025-10-17 22:47:59 | INFO | Historical EOD: using 2016-01-11
2025-10-17 22:47:59 | ERROR | 
1 Failed download:
2025-10-17 22:47:59 | ERROR | ['^CNX500']: YFTzMissingError('possibly delisted; no timezone found')
2025-10-17 22:48:00 | INFO | Using benchmark: ^CRSLDX
2025-10-17 22:48:01 | INFO | Positions -> portfolio_positions.csv | Trades -> trades_log.csv | Cash=500000.00
2025-10-17 22:48:01 | INFO | Done.
